### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url =  (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    # API results show 10 articles at a time
    page_query_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    #try: 
    # loop through the reviews["response"]["docs"] and append each review to the list
    for review in reviews['response']['docs']:
        reviews_list.append(review)
    # Print the page that was just retrieved
    print(f"{page} Page was checked")
    #except:
        # Print the page number that had no results then break from the loop
       # print(f"{page} was not check")
        #break

0 Page was checked
1 Page was checked
2 Page was checked
3 Page was checked
4 Page was checked
5 Page was checked
6 Page was checked
7 Page was checked
8 Page was checked
9 Page was checked
10 Page was checked
11 Page was checked
12 Page was checked
13 Page was checked
14 Page was checked
15 Page was checked
16 Page was checked
17 Page was checked
18 Page was checked
19 Page was checked


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:4],indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
#1:DataFrame and Column Reference: reviews_df["headline.main"] refers to a column named "headline.main" in the DataFrame reviews_df. This column contains string data.
#2:Lambda Function with String Manipulation: The apply() function is used to apply a lambda function to each element in the column "headline.main". A lambda function is a small, anonymous function defined using the keyword lambda.
#3String Slicing with Unicode Characters:
#st.find("\u2018"): This finds the index of the unicode character \u2018 in the string st. The unicode character \u2018 represents the ‘left single quotation mark’ (‘).
#st.find("\u2019 Review"): This finds the index of the substring \u2019 Review in the string st. Here, \u2019 is the unicode character for ‘right single quotation mark’ (’), and it is part of a larger substring that includes the word "Review".
#st[st.find("\u2018")+1:st.find("\u2019 Review")]: This slices the string st from just after the character ‘ to just before the substring ’ Review. 
#It effectively extracts the text between these markers.
#Assigning to a New Column: reviews_df["title"] = ... assigns the result of the apply() operation to a new column in reviews_df named "title".
#This new column will contain the extracted titles, assuming that the format of the strings in "headline.main" is consistent and includes these unicode quotation marks and the substring " Review".
#Output: The reviews_df at the end of the code indicates that the DataFrame is being returned or displayed after these operations. This DataFrame now includes the new "title" column with the extracted titles.

reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string


In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

title_list = reviews_df['title'].to_list()

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [13]:

tmdb_movies_list = []

request_counter = 0

for title in title_list:

    # Check sleep
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)  
    request_counter += 1    
    response = requests.get(
        f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}&language=en-US"
    )

    try:
        movie_id = response.json()['results'][0]['id']

        response_details = requests.get(
            f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US"
        )

        movie_details = response_details.json()
        genres = [genre['name'] for genre in movie_details['genres']]
        spoken_languages = [language['english_name'] for language in movie_details['spoken_languages']]
        production_countries = [country['name'] for country in movie_details['production_countries']]

        movie_data = {
            'title': title,
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        tmdb_movies_list.append(movie_data)

        print(f"Found details for: {title}")

    except IndexError:
        print(f"Movie not found: {title}")



Found details for: The Attachment Diaries
Movie not found: What’s Love Got to Do With It?’ Probably a Lo
Found details for: You Can Live Forever
Found details for: A Tourist’s Guide to Love
Found details for: Other People’s Children
Found details for: One True Loves
Found details for: The Lost Weekend: A Love Story
Found details for: A Thousand and One
Found details for: Your Place or Mine
Found details for: Love in the Time of Fentanyl
Found details for: Pamela, a Love Story
Found details for: In From the Side
Found details for: After Love
Found details for: Alcarràs
Found details for: Nelly & Nadine
Found details for: Lady Chatterley’s Lover
Found details for: The Sound of Christmas
Found details for: The Inspection
Found details for: Bones and All
Found details for: My Policeman
Found details for: About Fate
Found details for: Waiting for Bojangles
Found details for: I Love My Dad
Found details for: A Love Song
Found details for: Alone Together
Found details for: Art of Love
Found d

In [14]:
print("Number of results in tmdb_movies_list:", len(tmdb_movies_list))

Number of results in tmdb_movies_list: 154


In [15]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
    },
    {
        "title": "You Can Live Forever",
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English",
            "French"
        ],
        "production_countries": [
            "Canada",
            "United States of America"
        ]
    },
    {
        "title": "A Tourist\u2019s Guide to Love",
        "genres": [
            "Romance",
            "Comedy"
        ],
        "spoken_languages": [
            "English",
            "Vietnamese"
        ],
        "production_countries": [
            "United States of America"
        ]
    },
    {
        "title": "Other People\u2019s Children",
       

In [16]:
# Convert the results to a DataFrame

tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
print(tmdb_movies_df)

                         title                              genres  \
0       The Attachment Diaries  [Drama, Mystery, Thriller, Horror]   
1         You Can Live Forever                    [Drama, Romance]   
2    A Tourist’s Guide to Love                   [Romance, Comedy]   
3      Other People’s Children                     [Drama, Comedy]   
4               One True Loves            [Romance, Drama, Comedy]   
..                         ...                                 ...   
149              The Last Face                             [Drama]   
150              Lost in Paris                            [Comedy]   
151             The Other Half                    [Drama, Romance]   
152     The Ottoman Lieutenant               [Romance, Drama, War]   
153               Love & Taxes                   [Romance, Comedy]   

          spoken_languages                        production_countries  
0                [Spanish]                                 [Argentina]  
1        [Eng

### Merge and Clean the Data for Export

In [17]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_movies_df, on='title', how='inner')
print(merged_df)


                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/you-...   
2    https://www.nytimes.com/2023/04/21/movies/a-to...   
3    https://www.nytimes.com/2023/04/20/movies/othe...   
4    https://www.nytimes.com/2023/04/13/movies/one-...   
..                                                 ...   
149  https://www.nytimes.com/2017/07/27/movies/the-...   
150  https://www.nytimes.com/2017/06/15/movies/lost...   
151  https://www.nytimes.com/2017/03/09/movies/the-...   
152  https://www.nytimes.com/2017/03/09/movies/revi...   
153  https://www.nytimes.com/2017/03/02/movies/love...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Religion comes between two girls falling in lo...  The New York Times   
2    Rachael Leigh Cook stars in this bland rom-com...  The New York 

In [18]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', '\'']

# Loop through the list of columns to fix
for column in columns_to_fix:
    merged_df[column] = merged_df[column].astype(str)
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')
    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame
print(merged_df)

                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/you-...   
2    https://www.nytimes.com/2023/04/21/movies/a-to...   
3    https://www.nytimes.com/2023/04/20/movies/othe...   
4    https://www.nytimes.com/2023/04/13/movies/one-...   
..                                                 ...   
149  https://www.nytimes.com/2017/07/27/movies/the-...   
150  https://www.nytimes.com/2017/06/15/movies/lost...   
151  https://www.nytimes.com/2017/03/09/movies/the-...   
152  https://www.nytimes.com/2017/03/09/movies/revi...   
153  https://www.nytimes.com/2017/03/02/movies/love...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Religion comes between two girls falling in lo...  The New York Times   
2    Rachael Leigh Cook stars in this bland rom-com...  The New York 

In [19]:
# Drop "byline.person" column

merged_df.drop(columns=['byline.person'], inplace=True)
print(merged_df)


                                               web_url  \
0    https://www.nytimes.com/2023/05/25/movies/the-...   
1    https://www.nytimes.com/2023/05/04/movies/you-...   
2    https://www.nytimes.com/2023/04/21/movies/a-to...   
3    https://www.nytimes.com/2023/04/20/movies/othe...   
4    https://www.nytimes.com/2023/04/13/movies/one-...   
..                                                 ...   
149  https://www.nytimes.com/2017/07/27/movies/the-...   
150  https://www.nytimes.com/2017/06/15/movies/lost...   
151  https://www.nytimes.com/2017/03/09/movies/the-...   
152  https://www.nytimes.com/2017/03/09/movies/revi...   
153  https://www.nytimes.com/2017/03/02/movies/love...   

                                               snippet              source  \
0    A gynecologist and her patient form a horrifyi...  The New York Times   
1    Religion comes between two girls falling in lo...  The New York Times   
2    Rachael Leigh Cook stars in this bland rom-com...  The New York 

In [20]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)
print(merged_df)


TypeError: unhashable type: 'list'

In [21]:
# Export data to CSV without the index
merged_df.to_csv('merged_data.csv', index=False)